In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Reading Tweeter Data

In [6]:
dt_Australia= pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/Australia_tweet-ids.csv')
dt_Brazil= pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/Brazil_tweet-ids.csv')
dt_China = pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/China_tweet-ids.csv')
dt_Iran = pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/Iran_tweet-ids.csv')
dt_Italy = pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/Italy_tweet-ids.csv')
dt_Singapour = pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/Singapore_tweet-ids.csv')
dt_South_Korea= pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/SouthKorea_tweet-ids.csv')
dt_UK= pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/UK_tweet-ids.csv')
dt_USA= pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/1-  Tweets modifiers/Country Tweetsss/US_tweet-ids.csv')
Countries=[dt_Australia,dt_Singapour,dt_South_Korea,dt_UK,dt_USA]

In [7]:
import time
def Reading(Data): 
  data_tweet = pd.DataFrame()
  data_tweet['Time']= Data['created_at']
  data_tweet['text']= Data['text']
  data_tweet['user_followers_count']= Data['user_followers_count']
  data_tweet['user_followee_count']= Data['user_friends_count']
  data_tweet['lang']= Data['lang']
  return data_tweet



def date_correcting(data_tweet):
  OK=data_tweet['Time']
  df = pd.DataFrame()
  df['Time']=''
  for i in range(OK.shape[0]):
     X=OK[i]
     #Y= time.strftime('%Y%m%d%H', time.strptime(X,'%a %b %d %H:%M:%S +0000 %Y'))
     Y= time.strftime('%Y%m', time.strptime(X,'%a %b %d %H:%M:%S +0000 %Y'))
     df.at[i,'Time']=pd.to_numeric(Y)

  data_tweet.drop(['Time'], axis=1, inplace=True)
  data_tweet['time']=df['Time']
  data_tweet=data_tweet.sort_values(by=['time'])
  data_tweet2=data_tweet.copy()
  data_tweet.index=range(data_tweet.shape[0])
  return data_tweet

def en_correcting(data_tweet):
    a_list =[]
    for i in range(data_tweet.shape[0]):
      if data_tweet['lang'][i] != 'en':
          a_list.append(i) 
    data_tweet = data_tweet.drop(labels=a_list, axis=0)      
    data_tweet.index=range(data_tweet.shape[0])
    return  data_tweet

def process(Data):
  Data=Reading(Data)
  if Data.shape[0] >=80000:
     Data=Data.sample(n=80000)
  Data.index=range(Data.shape[0])
  Data=en_correcting(Data) 
  Data=Data.sample(n=43000)
  Data.index=range(Data.shape[0])
  Data=date_correcting(Data)
  return Data

In [8]:
Countries=[dt_Australia,dt_Singapour,dt_South_Korea,dt_UK,dt_USA]
for i in range(len(Countries)):  
   Countries[i]=process(Countries[i])

# Fake news detection

# Approach 2 ( Data Set 2):
 There are two files, one for real news and one for fake news (both in English) with a total of 23481 “fake” tweets and 21417 “real” articles.

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

In [9]:
fake = pd.read_csv("/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/3- Fake and real news detections+LIWC/Data_set 2/Fake.csv")
true = pd.read_csv("/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/3- Fake and real news detections+LIWC/Data_set 2/True.csv")
fake['target'] = 'FAKE'
true['target'] = 'REAL'
data = pd.concat([fake, true]).reset_index(drop = True)
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)

data.drop(["date"],axis=1,inplace=True)
data.drop(["title"],axis=1,inplace=True)
#lower case
data['text'] = data['text'].apply(lambda x: x.lower())
#remove punctuation
import string
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

data['text'] = data['text'].apply(punctuation_removal)

#Remove stopwords:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
x_train,x_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2, random_state=42)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,subject,target
0,donald trump administration white house leakie...,News,FAKE
1,washington reuters white house spokesman sean ...,politicsNews,REAL
2,almost 20 years ago david bowie released music...,News,FAKE
3,brussels reuters european union agreed financi...,worldnews,REAL
4,edinburgh reuters scotland first minister nico...,worldnews,REAL


In [12]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

 PassiveAggressiveClassifier?

Passive Aggressive algorithms are online learning algorithms. Such an algorithm remains passive for a correct classification outcome, and turns aggressive in the event of a miscalculation, updating and adjusting. Unlike most other algorithms, it does not converge. Its purpose is to make updates that correct the loss, causing very little change in the norm of the weight vector.


*Other Classifiers*

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
#part 2 
# Bagging meta-estimator #
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [23]:
classifiers = [#PassiveAggressiveClassifier(max_iter=50), 
               #KNeighborsRegressor(n_neighbors=10) , 
               #LogisticRegression(random_state=0,max_iter=100000),           
   #BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5),
    #KNeighborsClassifier(3),
    #SVC(kernel="linear", C=0.025),
    #SVC(gamma=2, C=1),
   #GaussianProcessClassifier(1.0 * RBF(1.0)),
    #DecisionTreeClassifier(max_depth=5),
    #RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    #AdaBoostClassifier(),
    #GaussianNB(),
    #QuadraticDiscriminantAnalysis()
    ]

classifiers_name = [ #'PassiveAggressiveClassifier(max_iter=50)',  
                    #'KNeighborsRegressor(n_neighbors=10)' ,   
                    #'logistic regression',
     #'BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5',             
    #'KNeighborsClassifier(3)',
    #'SVC(kernel="linear", C=0.025)',
    #'SVC(gamma=2, C=1)',
    #'GaussianProcessClassifier(1.0 * RBF(1.0))',
    #'DecisionTreeClassifier(max_depth=5)',
    #'RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)',
    ' Multi Layer Perceptron: MLPClassifier(alpha=1, max_iter=1000)',
    #'AdaBoostClassifier()',
    #'GaussianNB()/ Naive Bayes',
    #'QuadraticDiscriminantAnalysis()'
    ]
for i in range(0,len(classifiers)):
 print( classifiers_name[i])
 classifier=classifiers[i]
 classifier.fit(tfidf_train,y_train)
 y_pred = classifier.predict(tfidf_test)
 accuracy_met=metrics.accuracy_score(y_pred,y_test)
 print("Test Accuracy",accuracy_met)
 (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred, average='macro')
 print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
 print('confusion_matrix:',confusion_matrix(y_test,y_pred, labels=['FAKE','REAL']))
 print('\n ')


 Multi Layer Perceptron: MLPClassifier(alpha=1, max_iter=1000)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Test Accuracy 0.966369710467706
Precision: 0.9668706512820747	Recall: 0.9659245350500716	fscore:0.9662909745338478
confusion_matrix: [[4556  104]
 [ 198 4122]]

 


In [ ]:
classifier_PAC=PassiveAggressiveClassifier(max_iter=50) 
classifier_PAC.fit(tfidf_train,y_train)
classifier_LR=LogisticRegression(random_state=0,max_iter=100000)
classifier_LR.fit(tfidf_train,y_train)
classifier_DT=DecisionTreeClassifier(max_depth=5)
classifier_DT.fit(tfidf_train,y_train)
classifier_AB=AdaBoostClassifier()
classifier_AB.fit(tfidf_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

# Detecting Fake News  

In [ ]:
Australia_tweet=Countries[0]
#Brazil_tweet=pd.read_excel('/content/drive/Shareddrives/MY Files/Web scrapping/Data_set 1/Data.csv')
#China_tweet=pd.read_excel('/content/drive/Shareddrives/MY Files/Web scrapping/Data_set 1/Data.csv')
#Iran_tweet=pd.read_excel('/content/drive/Shareddrives/MY Files/Web scrapping/Data_set 1/Data.csv')
#Italy_tweet=pd.read_excel('/content/drive/Shareddrives/MY Files/Web scrapping/Data_set 1/Data.csv')
Singapore_tweet=Countries[1]
SouthKorea_tweet=Countries[2]
UK_tweet=Countries[3]
USA_tweet=Countries[4]
Countries_Tweet=[Australia_tweet,Singapore_tweet,SouthKorea_tweet,UK_tweet,USA_tweet]

In [ ]:
def labelling_ML(df):
    tfidf_train1=tfidf_vectorizer.transform(df['text'])
    df['Passive_Aggressive']=classifier_PAC.predict(tfidf_train1)
    df['Logistic_Regression']=classifier_LR.predict(tfidf_train1)
    df['Decision_Tree']=classifier_DT.predict(tfidf_train1)
    df['AdaBoost']=classifier_AB.predict(tfidf_train1)
    df = df.dropna(how='any',axis=0)
    df.index=list(range(df.shape[0]))
    return df

def labling_final(df):
   df['label']=''
   for i in range(df.shape[0]):
       n=0
       m=0
       if df['Passive_Aggressive'][i] == 'FAKE' :
               n=n+1
       else:
               m=m+1
       if df['AdaBoost'][i] == 'FAKE' :
               n=n+1
       else:
               m=m+1
       if df['Decision_Tree'][i] == 'FAKE' :
               n=n+1
       else:
               m=m+1
       if m < n:
               df.at[i,'label']='FAKE'    
       else:
               df.at[i,'label']='REAL'   
   return df

def real_n(df):
   n=0
   for i in range(df.shape[0]):
        if df['label'][i]=='REAL':
               n=n+1
   return n 
  
def seperation(df):
     fake_data=df[df['label']=='FAKE']
     real_data=df[df['label']=='REAL']
     return fake_data,real_data

#'label_PAC','label_LR','label_DT',"label_AB",'
def Monthly_fake(fake_data):
   fake_data2=fake_data.copy()
   fake_data3=fake_data.copy()
   fake_data.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'user_followers_count','user_followee_count'], axis=1, inplace=True)
   fake_data1 = fake_data.groupby(['time']).sum()
   fake_data2.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'text'], axis=1, inplace=True)
   fake_data2 = fake_data2.groupby(['time']).sum()
   fake_data3.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'text','user_followers_count','user_followee_count'], axis=1, inplace=True)
   data_tweet_frequency = fake_data3.pivot_table(index=['time'], aggfunc='size')
   fake_data3=fake_data3.groupby(['time']).sum()
   fake_data3['frequency']=data_tweet_frequency 
   ALL_Data_fake=fake_data1.merge(fake_data2, on='time')
   ALL_Data_fake=ALL_Data_fake.merge(fake_data3, on='time')
   return ALL_Data_fake

def Monthly_real(real_data):
    real_data2=real_data.copy()
    real_data3=real_data.copy()
    real_data.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'user_followers_count','user_followee_count'], axis=1, inplace=True)
    real_data1 = real_data.groupby(['time']).sum()
    real_data2.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'text'], axis=1, inplace=True)
    real_data2 = real_data2.groupby(['time']).sum()
    real_data3.drop(['lang','label','Passive_Aggressive','Logistic_Regression','Decision_Tree',"AdaBoost",'text','user_followers_count','user_followee_count'], axis=1, inplace=True)
    data_tweet_frequency = real_data3.pivot_table(index=['time'], aggfunc='size')
    real_data3=real_data3.groupby(['time']).sum()
    real_data3['frequency']=data_tweet_frequency 
    ALL_Data_real=real_data1.merge(real_data2, on='time')
    ALL_Data_real=ALL_Data_real.merge(real_data3, on='time')
    return ALL_Data_real


In [ ]:
#####################          Cleaning dataSET           #############################333

import nltk
nltk.download('stopwords')
import re
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
#from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer


def remove_html(text):
    """Remove special patterns - email, url, date etc."""
    email_regex = re.compile(r"[\w.-]+@[\w.-]+")
    url_regex = re.compile(r"(http|www)[^\s]+")
    date_regex = re.compile(r"[\d]{2,4}[ -/:]*[\d]{2,4}([ -/:]*[\d]{2,4})?") # a way to match date
    ## remove
    text = url_regex.sub("", text)
    text = email_regex.sub("", text)
    text = date_regex.sub("", text)
    return text



def remove_punctuation(text):
    no_punct="".join([c for c in text if c not in string.punctuation])
    return no_punct



#1 No tokenize
#tokenized_text=result2.lower()

#2 Tokenize
word_tokenizer = regexp.WhitespaceTokenizer()
#tokenized_text = [word.lower() for word in word_tokenizer.tokenize(result2)]


def remove_stopwords(text):
    words=[w for w in text if w not in stopwords.words('english')]
    return words

  
stemmer=PorterStemmer()
def word_stemmer(text):
    stem_text=" ".join([stemmer.stem(i) for i in text])
    return stem_text


def cleaning(ALL_Data_fake):  
  #ALL_Data_fake['texte_cleaned']=''
  for i in range(ALL_Data_fake.shape[0]): 
    j=ALL_Data_fake.index[i]
    result=ALL_Data_fake['text'][j]
    result1=remove_html(result)
    result2=remove_punctuation(result1)
    tokenized_text = [word.lower() for word in word_tokenizer.tokenize(result2)]
    result3=remove_stopwords(tokenized_text)
    result4=word_stemmer(result3)
    ALL_Data_fake.at[j,'texte_cleaned']=result4
  return ALL_Data_fake

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def fake_process(df):
     df=labelling_ML(df)
     df=labling_final(df)
     fake_data,real_data=seperation(df)
     fake_data.index=range(fake_data.shape[0])
     real_data.index=range(real_data.shape[0])
     fake_data= Monthly_fake(fake_data)
     real_data=Monthly_real(real_data)
     fake_data= cleaning(fake_data)
     real_data=cleaning(real_data)
     return fake_data,real_data

Countries_Tweet_fake=[Australia_tweet,Singapore_tweet,SouthKorea_tweet,UK_tweet,USA_tweet]
Countries_Tweet_real=[Australia_tweet,Singapore_tweet,SouthKorea_tweet,UK_tweet,USA_tweet]
Countries_Tweet_fake_name=['Australia_tweet_fake','Singapore_tweet_fake','SouthKorea_tweet_fake','UK_tweet_fake','USA_tweet_fake']
Countries_Tweet_real_name=['Australia_tweet_real','Singapore_tweet_real','SouthKorea_tweet_real','UK_tweet_real','USA_tweet_real']

for i in range(len(Countries_Tweet)):
     Countries_Tweet_fake[i],Countries_Tweet_real[i]=fake_process(Countries_Tweet[i])
     fake=Countries_Tweet_fake[i]
     real=Countries_Tweet_real[i] 
     fake_name=Countries_Tweet_fake_name[i]
     real_name=Countries_Tweet_real_name[i] 
     fake.to_excel(fake_name+'.xlsx',sheet_name='Sheet_name_1') 
     real.to_excel(real_name+'.xlsx',sheet_name='Sheet_name_1') 


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [ ]:
Countries_Tweet_fake[0].index=range(Countries_Tweet_fake[0].shape[0])
y=Countries_Tweet_fake[0]
#np.savetxt(r'c:\data\np.txt', np.array(y['texte_cleaned'][0]).value, fmt='%d')
#Countries_Tweet_fake[0].to_string('fff.txt') 